In [1]:
from games_setup import *
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph
import os
import networkx as nx

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
# following results from run_statistics
multimulti_errors = ['BIOMD0000000471_url.xml', 'BIOMD0000000468_url.xml',
'BIOMD0000000190_url.xml', 'BIOMD0000000572_url.xml',
'BIOMD0000000051_url.xml', 'BIOMD0000000479_url.xml',
'BIOMD0000000469_url.xml', 'BIOMD0000000175_url.xml',
'BIOMD0000000247_url.xml', 'BIOMD0000000049_url.xml',
'BIOMD0000000143_url.xml', 'BIOMD0000000470_url.xml',
'BIOMD0000000235_url.xml', 'BIOMD0000000052_url.xml',
'BIOMD0000000503_url.xml', 'BIOMD0000000426_url.xml',
'BIOMD0000000105_url.xml', 'BIOMD0000000167_url.xml',
'BIOMD0000000472_url.xml', 'BIOMD0000000245_url.xml',
'BIOMD0000000496_url.xml', 'BIOMD0000000473_url.xml',
'BIOMD0000000497_url.xml', 'BIOMD0000000248_url.xml',
'BIOMD0000000565_url.xml', 'BIOMD0000000364_url.xml',
'BIOMD0000000070_url.xml', 'BIOMD0000000232_url.xml',
'BIOMD0000000467_url.xml', 'BIOMD0000000383_url.xml',
'BIOMD0000000453_url.xml', 'BIOMD0000000112_url.xml',
'BIOMD0000000217_url.xml', 'BIOMD0000000452_url.xml',
'BIOMD0000000281_url.xml', 'BIOMD0000000439_url.xml',
'MODEL0072364382_url.xml', 'BIOMD0000000253_url.xml',
'BIOMD0000000243_url.xml', 'BIOMD0000000231_url.xml'
]

In [3]:
num = -3
simple = SimpleSBML()
simple.initialize(os.path.join(cn.DATA_DIR, multimulti_errors[num]))
m = MESGraph(simple)
m.analyze(simple.reactions)

No mass balance error found.
************************************
************************************
Unfortunately(?), We don't have type III errors


{Glc}->{HMP}
{HMP}->{Fru16P2}
{Fru16P2}->{ADP, ATP}
{ADP, ATP}->{HMP}
{ADP, ATP}->{Fru16P2}
{ADP, ATP}->{ADP, ATP}

In [4]:
for r in simple.reactions:
  if r.category == cn.REACTION_n_n:
    print(r.makeIdentifier(is_include_kinetics=False))

lower: Fru16P2 + 4.00 ADP -> 4.00 ATP


In [5]:
list(m.nodes)

[{Glc}, {HMP}, {Fru16P2}, {ADP, ATP}]

In [6]:
reaction = lower
som = m.getNode(simple.getMolecule('ADP'))
collections.deque([mole_stoich for mole_stoich in  
                          reaction.products if 
                       m.getNode(mole_stoich.molecule)==som])
#                          mole_stoich.molecule in som.molecules])

NameError: name 'lower' is not defined

In [7]:
lower = simple.getReaction('lower')
reduced_lower = m.reduceReaction(lower)
reduced_lower.reactants[0].stoichiometry

SOM =  {Glc}
(before) reactants_in deque([])
(before) products_in deque([])
(after) reactants_in deque([])
(after) products_in deque([])
SOM =  {HMP}
(before) reactants_in deque([])
(before) products_in deque([])
(after) reactants_in deque([])
(after) products_in deque([])
SOM =  {Fru16P2}
(before) reactants_in deque([Fru16P2 *  1.00])
(before) products_in deque([])
(after) reactants_in deque([Fru16P2 *  1.00])
(after) products_in deque([])
SOM =  {ADP, ATP}
(before) reactants_in deque([ADP *  4.00])
(before) products_in deque([ATP *  4.00])
(after) reactants_in deque([])
(after) products_in deque([])


1.0

In [11]:
reduced_lower.reactants

[Fru16P2 *  1.00]

In [13]:
num = -3
simple = SimpleSBML()
simple.initialize(os.path.join(cn.DATA_DIR, multimulti_errors[num]))
m = MESGraph(simple)
m.analyze(error_details=False)

Unfortunately(?), We don't have type III errors


{Glc}->{HMP}
{HMP}->{Fru16P2}
{Fru16P2}->{ADP, ATP}
{ADP, ATP}->{HMP}
{ADP, ATP}->{Fru16P2}
{ADP, ATP}->{ADP, ATP}

In [13]:
v2 = simple.getReaction('v2')
m.processMultiMultiReaction(v2)

True

In [15]:
m.type_three_errors

[]

In [125]:
def checkTypeThreeError(som1, som2, reaction):
  if m.has_edge(som1, som2):
    addTypeThreeError(som1, som2, reaction)
    return True
  elif m.has_edge(som2, som1):
    addTypeThreeError(som2, som1, reaction)
    return True
  return False

def mergeNodes(som1, som2, reaction):
  # Now mergeNodes assumes there is no error;
  # otherwise we would have already detected by checkTypeThreeError
  # (a) create a new SOM: som1.merge(som2)
  # (b) find edges starting or ending with som1/som2 and replace with this new SOM
  # delete som1 and som2 using remove_nodes_from
  new_som = som1.merge(som2)
  new_som.reactions.add(reaction)
  for som in [som1, som2]:
    for edge in list(m.in_edges(som)):
      remaining_som = edge[0]
      # add arc
      m.add_edge(remaining_som, new_som)  
    for edge in list(m.out_edges(som)):
      remaining_som = edge[1]
      # add arc
      m.add_edge(new_som, remaining_som) 
  m.remove_nodes_from([som1, som2])
  if not has_node(new_som):
    m.add_node(new_som)
  return m

In [78]:
def processMultiMultiReaction(reaction):
  pass
  # if all reactants are in one som, and all products are in one som
  # they must be the same (mergeSOM and update graph nodes & edges)
  flag = False
  reactants = [ms.molecule for ms in reaction.reactants]
  products = [ms.molecule for ms in reaction.products]
  reactant_soms = list({m.getNode(mole) for mole in reactants})
  product_soms = list({m.getNode(mole) for mole in products})
  print(reactant_soms)
  print(product_soms)
  if (len(reactant_soms)==1) and (len(product_soms)==1):
    flag = True
    # we should remove this multimulti reaction from self.multimulti_reactions
    reactant_stoichiometry = sum([ms.stoichiometry for ms in reaction.reactants])
    product_stoichiometry = sum([ms.stoichiometry for ms in reaction.products])
    if reactant_stoichiometry == product_stoichiometry:
        if reactant_soms[0] != product_soms[0]:
          if not checkTypeThreeError(som1, som2, reaction):
            mergeNodes(som1, som2, reaction)
        else:
          # this is obvious. pass (just add reaction)
          pass
    elif reactant_stoichciometry > product_stoichiometry:
      # meaning add arc from reactant_som -> product_som
      # check type I error and addArc
    else:
      # meaning add arc from product_som -> reactant_som
      # checck type I error and addArc
  else:
    # DON'T FORGET TO REMOVE THIS REACTION FROM SELF.MULTIMULTI_REACTIONS
    # now, we should reduce reaction
    # reduce reaction = namedtuple(label, reactants, products)
    # because of the above 
    # it doesn't merge soms but only add arcs (or type I errors)
    
    # flag = True if reduced reaction was used
    

  # elif, if both sides have the same som elements, reduce them by stoichiometry
  m.identifier = m.makeId()
  return flag

In [71]:
g = nx.DiGraph()
g.add_edges_from([('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'a'), ('a', 'e'), 
                  ('c', 'e'), ('f', 'c')])
# if remove 'a', all connected edges are deleted as well. But all nodes will still remain
g.remove_node('a')

In [72]:
print(g.edges)
print(g.nodes)

[('b', 'c'), ('c', 'd'), ('c', 'e'), ('f', 'c')]
['b', 'c', 'd', 'e', 'f']


In [73]:
a = [1, 2, 3, 4, 5]
a[1:] + [a[0]]

[2, 3, 4, 5, 1]

In [74]:
print("before", g.in_edges('c'))
for edge in list(g.in_edges('c')):
  remaining_som = edge[0]
  g.add_edge(remaining_som, 'z')
  g.remove_edge(edge[0], edge[1])
print("after", g.in_edges('z'))

before [('b', 'c'), ('f', 'c')]
after [('b', 'z'), ('f', 'z')]


In [75]:
g.edges

OutEdgeView([('b', 'z'), ('c', 'd'), ('c', 'e'), ('f', 'z')])

In [8]:
list(m.nodes)

[{Glc}, {HMP}, {Fru16P2}, {ADP, ATP}]